In [1]:
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.merge import merge

In [2]:
# all images wanted for the mosaic
files_to_mosaic = ['S2B_MSIL2A_20170519T101019_N0500_R022_T33UVU_20231115T232025_20m_bands_sentinel_clipped.tif',
             'S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025_20m_bands_sentinel_clipped.tif',
             'S2B_MSIL2A_20170519T101019_N0500_R022_T32UQD_20231115T232025_20m_bands_sentinel_clipped.tif']

# GT raster to clip the mosaic
polygon_path = r'C:\SKOLA\02_DOKTORAT\ml_dpz\lcz\berlin_data\berlin_polygon.shp'

##
polygon = gpd.read_file(polygon_path)

In [3]:
for f in files_to_mosaic:
    with rio.open(f) as src:
        print(f, src.crs)

S2B_MSIL2A_20170519T101019_N0500_R022_T33UVU_20231115T232025_20m_bands_sentinel_clipped.tif EPSG:32633
S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025_20m_bands_sentinel_clipped.tif EPSG:32633
S2B_MSIL2A_20170519T101019_N0500_R022_T32UQD_20231115T232025_20m_bands_sentinel_clipped.tif EPSG:32632


In [4]:
# nepotrebuju reproject
files_to_mosaic = ['S2B_MSIL2A_20170519T101019_N0500_R022_T33UVU_20231115T232025_20m_bands_sentinel_clipped.tif',
                   'S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025_20m_bands_sentinel_clipped.tif']


src_files_to_mosaic = []

#### Reproject

In [5]:
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import MemoryFile

# tento potrebuje reproject
need_reproject = ['S2B_MSIL2A_20170519T101019_N0500_R022_T32UQD_20231115T232025_20m_bands_sentinel_clipped.tif']
dst_crs = 'EPSG:32633'

for file_for_reproject in need_reproject:
    with rio.open(file_for_reproject) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
    
        with MemoryFile() as memfile:
            with memfile.open(**kwargs) as dst:  # Open as DatasetWriter
                for i in range(1, src.count + 1):
                    reproject(
                        source=rio.band(src, i),
                        destination=rio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
    
            reprojected_raster = memfile.open()
            src_files_to_mosaic.append(reprojected_raster)

#### Mosaic

In [6]:
for f in files_to_mosaic:
    src = rio.open(f)
    pixel_size = src.res
    pixel_size = int(round(pixel_size[0]))
    
    src_files_to_mosaic.append(src)

src_files_to_mosaic

[<open DatasetReader name='/vsimem/3ca52500-ff47-4127-ad3a-9c0c892c3943/3ca52500-ff47-4127-ad3a-9c0c892c3943.tif' mode='r'>,
 <open DatasetReader name='S2B_MSIL2A_20170519T101019_N0500_R022_T33UVU_20231115T232025_20m_bands_sentinel_clipped.tif' mode='r'>,
 <open DatasetReader name='S2B_MSIL2A_20170519T101019_N0500_R022_T33UVT_20231115T232025_20m_bands_sentinel_clipped.tif' mode='r'>]

In [7]:
mosaic, out_trans = merge(src_files_to_mosaic)

In [8]:
# export mosaic without clipping
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                })

with rio.open(f'{pixel_size}m_bands_mosaic.tif', "w", **out_meta) as dest:
    dest.write(mosaic)

In [8]:
# export mosaic with clipping the mosaic to the polygon extent
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                })

with MemoryFile() as memfile:
    with memfile.open(**out_meta) as dst:
        dst.write(mosaic)

    with memfile.open() as mosaic:
        clipped_image, clipped_transform = mask(mosaic, polygon['geometry'], crop=True)

        clipped_meta = mosaic.meta.copy()
        clipped_meta.update({
                "height": clipped_image.shape[1],
                "width": clipped_image.shape[2],
                "transform": clipped_transform
            })

        pixel_size = mosaic.res
        pixel_size = int(round(pixel_size[0]))
        
        with rio.open(f'{pixel_size}m_bands_mosaic.tif', "w", **clipped_meta) as dest:
            dest.write(clipped_image)